# 对齐 query 和文档

在 RAG 系统中，检索器可以利用单个嵌入模型来编码 query 和文档，用户的原始查询可能会受到措辞不精确和缺乏语义信息的影响，导致检索结果不符合预期。因此，将用户查询的语义空间与文档的语义空间保持一致至关重要。本节旨在介绍实现 query 和文档对齐的三个基本技术：query 改写、query 转换以及 Embedding 转换。

## 1. query 改写

### 1.1 定义
通过优化原始 query 的表述，使其更适合检索任务，解决用户原始 query 表达模糊或语义信息缺失的问题，从而提高检索的准确性和召回率。

### 1.2 实现方法
实现 query 改写的主要方法有：
* 使用语言模型改写 query：利用大语言模型对原始 query 进行改写，使其更加清晰、准确。例如，将复杂的自然语言问题转化为更简洁、更符合检索需求的表述。例如，将“我想知道关于人工智能的最新研究进展”改写为“人工智能最新研究论文”。
* 结合上下文信息改写 query：在多轮对话中，结合之前的对话内容对当前 query 进行改写，使当前 query 的语义信息更完整。例如，在用户询问“详细介绍这个算法的实现原理”时，结合历史对话提到的算法A，替换句中的指代对象，改写为“算法A的实现原理”。

### 1.3 代码实现

下面是一个结合对话历史进行 query 改写的代码示例，对话模型采用 `gpt-4o-mini`，Embedding 模型采用智谱 AI 的 `embedding-3`，向量数据库采用 `Chroma` 实现。

In [1]:
import os
import uuid
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_transformers import Html2TextTransformer
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from zhipuai_embedding import ZhipuAIEmbeddings

os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'
os.environ["HTTP_PROXY"] = 'http://127.0.0.1:7890'

从 `.env` 文件读取环境变量配置。

In [2]:
from dotenv import load_dotenv, find_dotenv

# 读取本地/项目的环境变量。

# find_dotenv() 寻找并定位 .env 文件的路径
# load_dotenv() 读取该 .env 文件，并将其中的环境变量加载到当前的运行环境中  
# 如果你设置的是全局的环境变量，这行代码则没有任何作用。
_ = load_dotenv(find_dotenv())

os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY') # OpenAI API key
os.environ['ZHIPUAI_API_KEY'] = os.getenv('ZHIPUAI_API_KEY') # ZHIPUAI API key

In [3]:
# 构造检索器，读取网页内容
loader = WebBaseLoader(
    web_paths=("https://movie.douban.com/subject/34780991/?from=showing",),
    encoding="utf-8", # 中文网页指定编码
)
docs = loader.load()

# html 转 text
html2test_transformer = Html2TextTransformer()
docs = html2test_transformer.transform_documents(docs)

# 分割文本
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
splits = text_splitter.split_documents(docs)

ids = [str(uuid.uuid4()) for _ in splits]

# embedding 模型
base_embeddings = ZhipuAIEmbeddings(
    api_key = os.environ['ZHIPUAI_API_KEY']
)

# 向量存储到 Chroma 数据库中
vectorstore = Chroma.from_documents(documents=splits, ids=ids, embedding=base_embeddings)
retriever = vectorstore.as_retriever()

In [4]:
docs

[Document(page_content='哪吒之魔童闹海 (豆瓣) 登录/注册 下载豆瓣客户端 豆瓣 6.0 全新发布 × 豆瓣 扫码直接下载 iPhone · Android 豆瓣 读书 电影 音乐 同城 小组 阅读 FM 时间 豆品 豆瓣电影 搜索： 影讯&购票 选电影 电视剧 排行榜 影评 2024年度榜单 2024年度报告 哪吒之魔童闹海 (2025) 导演: 饺子 编剧: 饺子 主演: 吕艳婷 / 囧森瑟夫 / 瀚墨 / 陈浩 / 绿绮 / 张珈铭 / 杨卫 / 王德顺 / 雨辰 / 李南 / 周泳汐 / 韩雨泽 / 南屿 / 张运气 / 杏林儿 / 王智行 / 张稷 / 良生 / 幻听 / 零柒 / 龚格尔 / 陈厚霖 / 饺子 类型: 剧情 / 喜剧 / 动画 / 奇幻 制片国家/地区: 中国大陆 语言: 汉语普通话 上映日期: 2025-01-29(中国大陆) 片长: 144分钟 又名: 哪吒2 / 哪吒2之魔童闹海 / Ne Zha 2 IMDb: tt34956443 豆瓣评分 引用 8.5 735698人评价 5星 46.3% 4星 37.3% 3星 13.3% 2星 2.4% 1星 0.8% 好于 95% 动画片 好于 98% 喜剧片 想看 看过 评价: 写短评 写影评 分享到 推荐 哪吒之魔童闹海的剧情简介 · · · · · · 天劫之后，哪吒、敖丙的灵魂虽保住了，但肉身很快会魂飞魄散。太乙真人打算用七色宝莲给二人重塑肉身。但是在重塑肉身的过程中却遇到重重困难，哪吒、敖丙的命运将走向何方？ 哪吒之魔童闹海的演职员 · · · · · · ( 全部 66 ) 饺子 导演 吕艳婷 配 儿童哪吒 囧森瑟夫 配 少年哪吒 / 结界兽左 瀚墨 配 敖丙 陈浩 配 李靖 绿绮 配 殷夫人 哪吒之魔童闹海的视频和图片 · · · · · · ( 预告片12 | 图片450 · 添加 ) 预告片 喜欢这部电影的人也喜欢 · · · · · · 哪吒3 长安三万里 8.3 心灵奇旅 8.7 天空之城 9.2 大闹天宫 9.4 哈尔的移动城堡 9.1 头脑特工队2 8.4 灌篮高手 8.9 蜘蛛侠：纵横宇宙 8.4 你想活出怎样的人生 7.5 我要写短评 哪吒之魔童闹海的短评 · · · · · · ( 全部 341249 条 ) 热门 / 最新 / 

In [5]:
# 定义改写用户提问的 prompt
contextualize_q_system_prompt = (
    "请根据给定的历史对话记录和用户提问，改写用户最终提出的问题。"
    "不要回答问题，只需要改写当前用户提问。"
    "没有对话历史则将用户最新的提问直接返回，有则进行改写。"
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
# 对话历史检索器
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [6]:
# 定义对话助手的 prompt
system_prompt = (
   "你是一个问答任务的助手。请根据检索到的上下文来回答问题。"
   "如果你不知道答案，就说不知道，不要随意编造答案。"
    "\n\n"
    "{context}"
)

# 创建问答 chain
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

# with history
store = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

rag 问答链构造完成后，就可以开始对话了。我们再自定义一个 session_id key，用于上下文串联，后续改写 query 时根据请求携带的 session_id 获取历史对话内容。

> 这里为了示例方便，我们使用的是简单固定的 session_id，对话记录也只是存储在一个简单的字典 store 中。实际应用时，一般需要给每个用户生成不同的唯一标识，并使用 MySQL、Redis 等数据存储服务管理每个用户的对话历史数据。

In [7]:
session_id = "abc123"
conversational_rag_chain.invoke(
    {"input": "介绍下《哪吒之魔童闹海》这部电影"},
    config={
        "configurable": {"session_id": session_id}
    },
)["answer"]

'《哪吒之魔童闹海》是《哪吒之魔童降世》的续作，由导演饺子执导。这部电影在视觉效果和特效上进行了全面升级，继续延续了前作的阴阳美学，展现了水与火、红与蓝、善与恶等元素的碰撞与交融。影片不仅在动作场面和变身效果上更加燃，更富有想象力，同时情感上也兼具笑与泪。\n\n影片的主题超越了对命运不公的反抗，探讨了许多小人物在既定约束下的挣扎与斗争。当个体的命运变成群体的命运时，影片对时代命运的洞察使得其内核更加深刻。整体来看，这部电影在故事构思、角色塑造和主题表达上都有所提升，受到了观众的广泛关注和讨论。'

在第一个问题基础上继续提问，使用代词“这部电影”指代前面提到的电影名称。

In [8]:
res = conversational_rag_chain.invoke(
    {"input": "这部电影的导演是谁"},
    config={"configurable": {"session_id": session_id}},
)
print("回答：\n" + res["answer"])

回答：
《哪吒之魔童闹海》的导演是饺子。


## 2. query 转换

### 2.1 HyDE

##### 2.1.1 概念介绍
HyDE（Hypothetical Document Embeddings）算法由 Gao 等人于 2023 年在[《Precise Zero-Shot Dense Retrieval without Relevance Labels》](https://arxiv.org/abs/2212.10496)论文中提出，背景是为了解决零样本密集检索的挑战，结合生成模型和嵌入模型，无需相关标签即可提升检索效果。

HyDE 即假设文档嵌入，旨在通过生成假设性文档来提升检索效果。其核心思想如下：
* 首先，在用户输入查询后，使用 LLM 在没有外部知识的情况下生成一个假设的、理想化的相关文档。生成的文档不一定是真实存在的，这一步的目的是捕捉查询的深层语义意图。
* 接下来，将这个假设性文档和原始查询都转换为嵌入，用于向量检索的相似性计算。
* 然后，从目标文档数据库中检索，找到在向量空间中最接近这些嵌入的真实文档。

![HyDE](./figures/HyDE.png)

#### 2.1.2 优势和局限性

区别于直接使用 query 来检索嵌入相似性，HyDE 是检索从一个答案到另一个答案的嵌入相似性，可以让根据 query 生成的假设性文档和目标文档的语义空间保持一致。通过生成假设文档，HyDE 能覆盖更广泛的语义关联，具备更好的泛化能力；并且无需依赖标注数据，具有零样本（Zero-Shot）适应能力。然而，HyDE 方法依赖生成模型的可靠性，若模型本身存在偏见或知识过时，最终检索结果可能会不理想。

#### 2.1.3 代码实现

In [9]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# 定义 HyDE 生成答案的 prompt
template = """请根据以下问题给出你的回答。

问题: {question}

回答:
"""

prompt_hyde = ChatPromptTemplate.from_template(template)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
output_parser = StrOutputParser()

retrieval_hyde = (
    prompt_hyde | llm | output_parser
)

question = "电影哪吒之魔童降世讲述了什么故事？"
retrieval_hyde.invoke({"question":question})


'《哪吒之魔童降世》讲述了一个关于成长与自我认同的故事。影片围绕着中国传统神话中的哪吒角色展开，讲述了他从一个被视为“魔童”的孩子，逐渐成长为英雄的过程。\n\n故事的背景设定在一个神话世界，哪吒是莲花化身的灵魂，出生时被预言为将给人间带来灾难，因此遭到众人的误解和排斥。面对外界的偏见和压力，哪吒内心挣扎，但他并不甘心被命运所左右，努力寻找自己的价值和归属。\n\n在与父母、朋友以及敌人之间的互动中，哪吒逐渐认识到真正的勇气和责任，并最终选择保护他所爱的人，打破命运的束缚，成为了一个真正的英雄。影片通过哪吒的成长历程，传达了关于勇敢做自己、打破偏见和追求自由的重要主题。'

In [10]:
# 将上述生成的回答，作为输入到向量库中进行检索。
# 这里继续沿用前一小节已导入数据的向量库。
retrieval_chain = retrieval_hyde | retriever 
retireved_docs = retrieval_chain.invoke({"question": question})

In [11]:
# 查看根据 HyDE 回答检索到的文档内容
retireved_docs

[Document(page_content='致父母之辈 这篇影评可能有剧透 看完走出电影院 问妈妈：好看吗 妈妈：好看 过了两分钟 妈妈：这电影是给你们这辈看的 也是给我们这辈看的 最聪明的妈妈即刻拿捏精髓 对父母来讲 我们都是魔丸 别人家的小孩纵然是灵珠 无论如何 他们爱的还是我们 虽然说失去了才知道珍惜是句名言 可导演还是在努力告诉我们 不... (展开) 490 47 59回应 收起 徐旷来 2025-01-26 08:42:49 哪吒最早出自哪？《哪吒之魔童闹海》 哪吒，一听这名字的读音，就不像汉语。 哪吒从何而来？从印度来。古人翻译佛经，有在汉字左边加口字的习惯，比如压了孙猴子五百年的五指山上的梵文六字咒语：唵（ōng）、嘛（ma）、呢（nī）、叭（bēi）、咪（mēi）、吽（hōng），意味着清除贪、嗔、痴、傲慢、嫉妒、吝啬。... (展开) 355 35 50回应 收起 导师永远的战士 2025-01-29 16:16:01 合格的商业片，但仍想提出质疑 作为贺岁档电影，哪吒无疑是一部合格的商业片。故事节奏快，至少看着不会无聊，围绕亲情线展开，也适合全家一起看。特效场面宏大，一看便知是烧了钱的，也能值回票价来。如果是专注磕cp的无脑磕药鸡，作品也满足要求大卖特卖，两男主几乎时时黏在一起，包含同人女喜欢的两个灵... (展开) 494 107 168回应 收起 > 更多影评 3812篇 发起新的讨论 小组讨论 · · · · · · 哪吒1我5刷（不算流媒体），哪吒2却没有2刷的冲动 来自李先生 26 回应 2025-02-06 19:13:41 报一下军情，已经60亿了 来自重庆猛男 1 回应 2025-02-06 19:13:18 郭靖已死，哪吒出世——从“英雄觉醒”的角度比较... 来自光影书编 17 回应 2025-02-06 19:12:53 不谈麦麸和女性主义，就纯内容角度谈谈有多烂总可... 来自云霁 696 回应 2025-02-06 19:12:26 2d，3d，imax影厅区别大吗 来自豆友AAjv_Hc2so 5 回应 2025-02-06 19:12:21 🍐🍑，客观评价《哪吒2》暨拆穿黑子的假面 来自踏风归来 67 回应 2025-02-06 19:12:12 现在的电影也就赢在营销宣传上了 来自可爱☺ 63 回应 2025

In [12]:
template = """根据以下内容回答用户提问:

{context}

用户提问: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"context":retireved_docs, "question":question})

'《哪吒之魔童降世》讲述了一个关于命运与自我认同的故事。影片围绕主角哪吒展开，他是一个被认为是“魔”的孩子，出生时就注定要与命运抗争。故事中，哪吒面临着来自外界的偏见和压力，但他努力寻找自己的身份，试图打破宿命的束缚。\n\n影片通过哪吒的成长历程，探讨了亲情、友情和自我救赎的主题。哪吒在与敌人斗争的过程中，不仅要面对外部的挑战，还要克服内心的挣扎，最终实现了自我救赎，证明了自己并不是命运所定义的“魔”，而是可以选择自己道路的英雄。\n\n整部电影结合了中国传统文化元素与现代价值观，展现了对命运不公的反抗和对自我价值的追求，深受观众喜爱。'

### 2.2 Step-Back Prompting

#### 2.2.1 概念介绍

Step-Back Prompting 由 Google DeepMind 团队于 2024 年 3 月在[《Take a Step Back: Evoking Reasoning via Abstraction in Large Language Models》](https://arxiv.org/pdf/2310.06117)论文中提出，是一种旨在提升 LLM 复杂问题推理能力的提示技术，其核心思想是引导模型进行抽象思考，从具体问题中抽象出高层次的概念或原理，从而增强对复杂问题的推理能力。具体策略如下：
* 首先使用 LLM "后退一步"生成高层次的抽象概念，将推理建立在抽象概念的基础上，以减少在中间推理步骤中出错的概率；
* 然后将抽象的概念和原始问题都用来进行检索，并把检索到的结果都用来作为 LLM 响应的基础。

![Step-Back Prompting](./figures/Step-Back-Prompting.png)

例如，上述图片中的一个例子是，"Estella Leopold 在 1954 年 8 月至 1954 年 11 月期间去了哪所学校？"，这类问题对于 LLM 来说很容易答错，因为文档中可能并没有直接相关的内容。但是如果后退一步，站在更高层次对问题进行抽象，提出一个新的问题："Estella Leopold 的教育背景是什么？"，那 LLM 可以先将 Estella Leopold 的教育经历都列出来，然后将这些信息和原始问题放在一起，这样对于 LLM 来说就可以很容易给出正确答案。

#### 2.2.2 优势和局限性

Step-Back Prompting 通过 "抽象化->具体推理" 的分层策略，能够提升大模型在复杂任务中的表现，但其效果也高度依赖模型本身的领域知识和抽象能力。

#### 2.2.3 代码实现

In [13]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI

# 提供 Few Shot 示例，用于指导模型如何回答问题
examples = [
    {
        "input": "中国农历年腊月三十日放假吗？",
        "output": "中国农历假期时间有哪些？",
    },
    {
        "input": "李白是哪个朝代的诗人？",
        "output": "李白的生平简介?",
    },
]
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# 定义系统提示和最终的 Prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """你是一个擅长将问题转化为更通用问题的专家。你的任务是将具体的问题转化为更抽象的、更通用的“退一步”问题，以便更容易回答。以下是一些示例：""",
        ),
        # Few shot 示例
        few_shot_prompt,
        # 用户输入的原始问题
        ("user", "{question}"),
    ]
)

In [14]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
output_parser = StrOutputParser()
question_gen = prompt | llm | output_parser

In [15]:
question = "农历正月初五是假期吗？"

In [16]:
question_gen.invoke({"question": question})

'农历新年期间的假期安排是怎样的？'

In [17]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

search = DuckDuckGoSearchAPIWrapper(backend='auto', max_results=4)

def retriever(query):
    return search.run(query)

In [18]:
retriever(question)

'国务院办公厅印发《关于2025年部分节假日安排的通知》。 经党中央、国务院批准，根据2024年11月修订的《全国年节及纪念日放假办法》，自2025年1月1日起，全体公民放假的假日增加2天，其中春节、劳动节各增加1天。. 除个别特殊情形外，春节自农历除夕起放假调休8天，国庆节自10月1日起放假调休7 ... 经党中央、国务院批准，根据2024年11月修订的《全国年节及纪念日放假办法》，自2025年1月1日起，全体公民放假的假日增加2天，其中春节、劳动节各增加1天。据此对放假调休原则作进一步优化完善，除个别特殊情形外，春节自农历除夕起放假调休8天，国庆节自10月1日起放假调休7天，劳动节放假 ... 一、将第二条第二项修改为"（二）春节，放假4天（农历除夕、正月初一至初三）"，第四项修改为"（四）劳动节，放假2天（5月1日、2日 ... 经党中央、国务院批准，根据2024年11月修订的《全国年节及纪念日放假办法》，自2025年1月1日起，全体公民放假的假日增加2天，其中春节、劳动节各增加1天。 ... 二、春节：1月28日（农历除夕、周二）至2月4日（农历正月初七、周二）放假调休，共8天。1月26日 ...'

In [20]:
response_prompt_template = """你是一名权威的知识领域专家，请你根据以下内容回答用户提问。如果用户提问与以下内容无关，则请直接忽略这些内容。

{normal_context}
{step_back_context}

用户提问: {question}
回答:"""
response_prompt = ChatPromptTemplate.from_template(response_prompt_template)

chain = (
    {
        # Retrieve context using the normal question
        "normal_context": RunnableLambda(lambda x: x["question"]) | retriever,
        # Retrieve context using the step-back question
        "step_back_context": question_gen | retriever,
        # Pass on the question
        "question": lambda x: x["question"],
    }
    | response_prompt
    | llm
    | output_parser
)

chain.invoke({"question": question})


'是的，农历正月初五在2025年的春节假期中是一个假期。根据国务院办公厅的通知，春节假期从1月28日（农历除夕）开始，到2月4日（农历正月初七）结束，共计8天。因此，农历正月初五（2月2日）也是假期的一部分。'

### 2.3 子查询

#### 2.3.1 概念介绍

子查询使用分而治之的方法来处理复杂的问题，其核心思想是在问答过程中生成并提出与主问题相关的子问题，以便更好地理解和回答主问题。这些子问题通常更具体，可以帮助系统更深入地理解主问题，从而提高检索准确性和提供正确的答案。具体步骤如下：

* 查询分解：利用大语言模型将用户的原始查询分解为多个子查询。
* 检索阶段：针对每个子查询，检索模块在知识库或文档集合中检索相关的文档。
* 合并与生成：将检索到的文档集拼接起来，再由大语言模型根据这些上下文信息生成最终答案。

#### 2.3.2 优势和局限性
子查询通过将复杂问题分解为多个子问题，可以更精准地检索到与每个子问题相关的文档，从而提高整体检索的准确性和相关性。并且由于子查询可以覆盖原始查询的多个方面，生成的答案也更加丰富和准确。
但同时也存在一些局限性，例如过多的子查询可能增加系统的计算负担和延迟，最终的效果也依赖检索模块的性能。

## 参考
* [https://python.langchain.com/v0.2/docs/tutorials/qa_chat_history/](https://python.langchain.com/v0.2/docs/tutorials/qa_chat_history/)
* [https://github.com/langchain-ai/langchain/blob/master/cookbook/hypothetical_document_embeddings.ipynb](https://github.com/langchain-ai/langchain/blob/master/cookbook/hypothetical_document_embeddings.ipynb)